In [ ]:
%load_ext autoreload
%autoreload 2

## Libraries

# Variable factory

In [3]:
import sys
# Add root to path
sys.path.append("..")

In [4]:
from typing import Literal
import ee
from component.script.gee.ee_fao_gaul import get_fao_gaul_features, get_fao_gaul_subj
from component.script.variables.models import RasterizationMethod
from component.script.variables import LocalVectorVar, LocalRasterVar, GEEVar
from component.script.project import Project
from component.script.gee.ee_rasterize_unique_values import gee_rasterize_unique_values
from component.script.project import Project


## GEE


In [5]:
ee_project = "ee-dfgm2006"
ee.Initialize(project=ee_project)

In [6]:
image = ee.Image(1)
forest_change_var_t1 = GEEVar(
    name=f"test",
    data_type="raster",
    gee_images=[image],
)

## Set project parameters

In [7]:
project_name = "nuevo3"
project = Project(project_name=project_name)

In [ ]:
# Load the project from JSON
# project = Project.load(project_name=project_name)

In [ ]:
# Check the project variables (have to be empty)
project.variables

In [ ]:
# We can save the project
# project.save()

In [ ]:
project

# Create an area of interest

In [ ]:
aoi_source: Literal["gee", "local"] = "local"

## Use GEE

In [ ]:
if aoi_source=="gee":

    iso_code = "MTQ"
    aoi_image = get_fao_gaul_features(level=0, code=iso_code)

    aoi_var = GEEVar(
        name="aoi",
        data_type="vector",
        gee_images=[aoi_image],
        project=project,
        aoi=aoi_image.geometry(),
    )

    # Download and convert to local vector
    aoi_local = aoi_var.to_local_vector()
    aoi = aoi_image


else:
    aoi_local_var = LocalVectorVar(
        path="/home/dguerrero/1_modules/deforisk-jupyter-nb-v2/data/Datos_pruebas_degradacion/Limite_SabanasYari/Limite_SabanasYari/NAD_sabanasYari.shp",
        name="aoi",
        project=project,
    )

    aoi = aoi_local_var.to_gee_var()

## SubJuridistion

In [ ]:
filtered_subj, filtered_attribute = get_fao_gaul_subj(2, aoi)
subj_image = (
    ee.Image(gee_rasterize_unique_values(filtered_subj, "gaul2_name"))
    .clip(aoi)
    .toByte()
)

subj_var = GEEVar(
    name="subj",
    data_type="raster",
    raster_type="categorical",
    gee_images=[subj_image],
    aoi=aoi,
    project=project,
    # tags=["forest_loss"]

)
subj_var_local = subj_var.to_local_raster(overwrite=True) # Returns a LocalVar instance
subj_var_local.add_as_raw();

# GEE VARIABLES

## Areas protegidas

In [ ]:
wdpa_poly = (
    ee.FeatureCollection("WCMC/WDPA/current/polygons")
    .filterBounds(aoi)
    .filter(
        ee.Filter.inList(
            "STATUS", ["Designated", "Inscribed", "Established", "Proposed"]
        )
    )
)
wdpa_image = (
    wdpa_poly.reduceToImage(["WDPAID"], ee.Reducer.first())
    .gt(0)
    .unmask()
    .clip(aoi)
    .toByte()
)

protected_area_var = GEEVar(
    name="protected_area",
    data_type="raster",
    raster_type="categorical",
    gee_images=[wdpa_image],
    aoi=aoi,
    project=project
)

protected_area_var_local = protected_area_var.to_local_raster()
protected_area_var_local.add_as_raw();

## Altitude

In [ ]:
altitude_image = ee.Image("USGS/SRTMGL1_003").select("elevation").clip(aoi)
altitude_var = GEEVar(
    name="altitude",
    data_type="raster",
    raster_type="continuous",
    gee_images=[altitude_image],
    aoi=aoi,
    project=project
)
altitude_var_local = altitude_var.to_local_raster()
altitude_var_local.add_as_raw();

## Slope

In [ ]:
slope_image = ee.Terrain.slope(altitude_image).clip(aoi)
slope_var = GEEVar(
    name="slope",
    data_type="raster",
    raster_type="continuous",
    gee_images=[slope_image],
    aoi=aoi,
    project=project
)
slope_var_local = slope_var.to_local_raster()
slope_var_local.add_as_raw();

## Forest layers

In [ ]:
forest_source = "gfc"  # "gfc" or "tmf"
tree_cover_threshold = 10  # in percentage

# Define the years you want to work with
years = [2015, 2020, 2024]

if forest_source == "gfc":
    gfcImage = ee.Image("UMD/hansen/global_forest_change_2024_v1_12")
    clipGfc = gfcImage.clip(aoi)
    forest2000 = clipGfc.select(["treecover2000"])
    forest2000_thr = (
        ee.Image(0).where(forest2000.gte(tree_cover_threshold), 1).clip(aoi)
    )
    loss = clipGfc.select(["lossyear"])

    forest_gcf_t1 = forest2000_thr.where(loss.lt(years[0] - 2000), 0).rename("B1")
    forest_gcf_t2 = forest2000_thr.where(loss.lt(years[1] - 2000), 0).rename("B1")
    forest_gcf_t3 = forest2000_thr.where(loss.lt(years[2] - 2000), 0).rename("B1")

    # Create forest variables with new structure
    forest_images = [forest_gcf_t1, forest_gcf_t2, forest_gcf_t3]
    forest_change_vars = []
    
    for year, forest_img in zip(years, forest_images):
        forest_var = GEEVar(
            name="forest_gfc",
            data_type="raster",
            raster_type="categorical",
            gee_images=[forest_img],
            aoi=aoi,
            project=project,
            year=year,
            tags=["forest"]
        )
        forest_change_vars.append(forest_var)

elif forest_source == "tmf":
    tmfImage = (
        ee.ImageCollection("projects/JRC/TMF/v1_2024/AnnualChanges")
        .filterBounds(aoi)
        .mosaic()
    )
    forest2_t1 = tmfImage.select("Dec" + str(years[0] - 1))
    forest2_t2 = tmfImage.select("Dec" + str(years[1] - 1))
    forest2_t3 = tmfImage.select("Dec" + str(years[2] - 1))

    forest_tmf_t1 = (
        forest2_t1.where(forest2_t1.eq(2), 1).where(forest2_t1.neq(1), 0).rename("B1")
    )
    forest_tmf_t2 = (
        forest2_t2.where(forest2_t2.eq(2), 1).where(forest2_t2.neq(1), 0).rename("B2")
    )
    forest_tmf_t3 = (
        forest2_t3.where(forest2_t3.eq(2), 1).where(forest2_t3.neq(1), 0).rename("B3")
    )

    # Create forest variables with new structure
    forest_images = [forest_tmf_t1, forest_tmf_t2, forest_tmf_t3]
    forest_change_vars = []
    
    for year, forest_img in zip(years, forest_images):
        forest_var = GEEVar(
            base_name="forest_tmf",  # Clean base name
            name="forest_tmf",
            data_type="raster",
            raster_type="categorical",
            gee_images=[forest_img],
            aoi=aoi,
            project=project,
            year=year,
            tags=["forest"]
        )
        forest_change_vars.append(forest_var)

# Convert each GEEVar to local and add as raw
for forest_change_var in forest_change_vars:
    forest_change_var_local = forest_change_var.to_local_raster()
    forest_change_var_local.add_as_raw()


## Rivers

In [ ]:
rivers_image = (
    ee.ImageCollection("projects/sat-io/open-datasets/OSM_waterLayer")
    .filterBounds(aoi)
    .mosaic()
    .clip(aoi)
)
rivers_image = rivers_image.gte(2).unmask().clip(aoi).toByte()
rivers_var = GEEVar(
    name="rivers",
    data_type="raster",
    raster_type="categorical",
    gee_images=[rivers_image],
    aoi=aoi,
    project=project
)

rivers_var_local = rivers_var.to_local_raster()
rivers_var_local.add_as_raw();

## Roads

In [ ]:
roads_image = (
    ee.Image(
        "projects/ee-andyarnellgee/assets/crosscutting/infrastructure/roads_osm/roadsAllImageOSM"
    )
    .unmask()
    .clip(aoi)
    .toByte()
)
roads_var = GEEVar(
    name="roads",
    data_type="raster",
    raster_type="categorical",
    gee_images=[roads_image],
    aoi=aoi,
    project=project

)

roads_var_local = roads_var.to_local_raster()
roads_var_local.add_as_raw();

## Towns

In [ ]:
jrc_ghsl_pop = ee.ImageCollection("JRC/GHSL/P2023A/GHS_POP")

def closest_epoch(year):
    # Define the list of epochs from 1975 to 2020 with 5-year intervals
    epochs = list(range(1975, 2021, 5))

    # Find the closest epoch to the given year
    closest = min(epochs, key=lambda x: abs(x - year))

    return closest


# Apply the function to each year
closest_epochs = [closest_epoch(y) for y in years]


cities_pop_1 = ee.Image("JRC/GHSL/P2023A/GHS_POP/" + str(closest_epochs[0]))
cities_pop_2 = ee.Image("JRC/GHSL/P2023A/GHS_POP/" + str(closest_epochs[1]))
cities_pop_3 = ee.Image("JRC/GHSL/P2023A/GHS_POP/" + str(closest_epochs[2]))

cities_build_1 = ee.Image(
    "JRC/GHSL/P2023A/GHS_BUILT_S/" + str(closest_epochs[0])
).select("built_surface")
cities_build_2 = ee.Image(
    "JRC/GHSL/P2023A/GHS_BUILT_S/" + str(closest_epochs[1])
).select("built_surface")
cities_build_3 = ee.Image(
    "JRC/GHSL/P2023A/GHS_BUILT_S/" + str(closest_epochs[2])
).select("built_surface")

cities1 = (
    ee.Image(0).where(cities_pop_1.gte(15).And(cities_build_1.gte(90)), 1).clip(aoi)
)
cities2 = (
    ee.Image(0).where(cities_pop_2.gte(15).And(cities_build_2.gte(90)), 1).clip(aoi)
)
cities3 = (
    ee.Image(0).where(cities_pop_3.gte(15).And(cities_build_3.gte(90)), 1).clip(aoi)
)

# Create separate GEEVar for each time period - explicit and clear
towns_var_t1 = GEEVar(
    name="towns",
    data_type="raster",
    raster_type="categorical",
    gee_images=[cities1],
    aoi=aoi,
    project=project,
    year=years[0],
    tags=["town"]
)

towns_var_t2 = GEEVar(
    base_name="towns",  # Clean base name for temporal grouping
    name="towns",
    data_type="raster",
    raster_type="categorical",
    gee_images=[cities2],
    aoi=aoi,
    project=project,
    year=years[1],
    tags=["town"]
)

towns_var_t3 = GEEVar(
    base_name="towns",  # Clean base name for temporal grouping
    name="towns",
    data_type="raster",
    raster_type="categorical",
    gee_images=[cities3],
    aoi=aoi,
    project=project,
    year=years[2],
    tags=["town"]
)

towns_vars = [towns_var_t1, towns_var_t2, towns_var_t3]

# Convert each GEEVar to local and add as raw
for towns_var in towns_vars:

    towns_var_local = towns_var.to_local_raster()
    towns_var_local.add_as_raw()

In [ ]:
## Consider using Global Human Modification v3
# https://gee-community-catalog.org/projects/ghm/?h=human

## Oxford accessibility to cities 2015
## ee.Image('Oxford/MAP/accessibility_to_cities_2015_v1_0')


## Custom variables

### GEEVars

In [ ]:
# Example

new_var = GEEVar(
    name="varname",
    data_type="raster",
    gee_images=[],
    aoi=aoi,
    project=project
)
new_var

### LocalVector

In [ ]:
local_centros_poblados_raster_dist = LocalVectorVar(
    name="centros_poblados",
    path="/home/dguerrero/1_modules/deforisk-jupyter-nb-v2/data/Datos_pruebas_degradacion/capas_crudo/asentamientos/Centrospoblados.shp",
    project=project,
    rasterization_method=RasterizationMethod.binary
).add_as_raw()

### LocalRaster

In [ ]:
from component.script.variables.models import RasterType


coca_var = LocalRasterVar(
    name="coca",
    path="/home/dguerrero/1_modules/deforisk-jupyter-nb-v2/data/Datos_pruebas_degradacion/Variables_raster/DE_COCA.img",
    project=project,
    raster_type=RasterType.categorical
)

In [ ]:
#TODO: make an example